In [1]:
import os
import pandas as pd
import shutil
columnNames = ["FolderName","Location","OriginalFileName","PrefixIndex"]
fileExcludeList = ['playground.ipynb','example.xlsx','scriptLog.xlsx']
scriptLogFileName = 'scriptLog.xlsx'

In [2]:
# # import xlsxwriter

# workbook = xlsxwriter.Workbook('scriptLog.xlsx')
 
# # The workbook object is then used to add new
# # worksheet via the add_worksheet() method.
# worksheet = workbook.add_worksheet()
 
# # Use the worksheet object to write
# # data via the write() method.
# worksheet.write('A1', 'FolderName')
# worksheet.write('B1', 'Location')
# worksheet.write('C1', 'OriginalFileName')
# worksheet.write('D1', 'PrefixIndex')
 
# # Finally, close the Excel file
# # via the close() method.
# workbook.close()

In [3]:
#wrapper for the os.listdir function since its used frequently.
def getCurrentFileList():
    currentFiles = os.listdir('.')
    return(currentFiles)

In [4]:
#function that generates the log file, excludes all files starting with '.', '~' and all the files in the fileExcludeList
#if exp is found in name => assumped to be the pdf of the entry and puts it at top with a default folder name
#default folder name replaced "." with spaces and removes .pdf
def generateLogFile(df):
    fileList = getCurrentFileList()
    for file in fileList:
    #     print(file)
        if file[0] != '.' and file[0] != '~' and file not in fileExcludeList:
            if "EXP" in file:
                folderName = file.strip('.pdf').replace('.',' ').replace('-',' ')
                appendRowToDF(df,[folderName,'',file,'0'],'top')
            else:
                appendRowToDF(df,['','',file,''],'bot')
    print(f"{scriptLogFileName} Generated")
    return(df)

In [5]:
def doesLogFileExist():
    fileList = getCurrentFileList()
    for file in fileList:
    #     print(file)
        if file == scriptLogFileName:
            print(f"{scriptLogFileName} File Exsists,moving onto Reading and relabeling")
            return(True)
    print(f"{scriptLogFileName} not found, First Execution in folder")
    df = pd.DataFrame(columns=columnNames)
    df = generateLogFile(df)
    df.to_excel(scriptLogFileName)
    return(False)

In [6]:
def appendRowToDF(df,infoObj,position="bot"):
    if position == 'top':
        df.loc[-1] = infoObj
        df.index = df.index + 1
        df.sort_index(inplace=True)
    else:
        df.loc[df.shape[0]] = infoObj
    return(df)
    

In [7]:
def formatNumberToAppend(number):
    try:
        if int(number) < 10:
            return(f'0{int(number)}')
        if int(number) >= 10:
            return(f"{int(number)}")
    except AttributeError:
        return(number)
    except ValueError:
        return(number)

In [8]:
def copyandRename():
    df = pd.read_excel(scriptLogFileName)
    df["FormatedPreFix"] = df["PrefixIndex"].map(formatNumberToAppend)
    df["RenamedFileNames"] = df["FormatedPreFix"] +'_'+ df['OriginalFileName']
    locationValue = df['Location'][0]
    if str(locationValue) == 'nan':
        location = os.getcwd()
    else:
        location = df['Location'][0]
    print(f"Location of folder: {location}")
    folderName = str(df['FolderName'][0])
    try:
        folderlocation = os.path.join(location,folderName)
        os.mkdir(folderlocation)
    except OSError as error:
        print(error)
    for pos in df.index:
    #     print(df.iloc[pos])
        currentFileName = df["OriginalFileName"][pos]
        renamedFileName = df["RenamedFileNames"][pos]
        if str(renamedFileName) == 'nan':
            print(f"Skipping {currentFileName}, since no PrefixIndex was assigned ")
        else:    
            print(currentFileName,renamedFileName)
            src = os.path.join(os.getcwd(),currentFileName)
            dst = os.path.join(location,folderName,renamedFileName)
            shutil.copy(src,dst)
            print(f"""
            Original
            Location : {src}
            Name : {currentFileName}

            Renamed
            Location: {dst}
            Name : {renamedFileName}
            ###########################
            """)


In [9]:
logStatus = doesLogFileExist()
if logStatus:
    print("Time to relabel the files and generate the new files")
    copyandRename()
else:
    print(f"First Execution, generated {scriptLogFileName}, Please map your Prefix and rerun to generate the relabed files.")

scriptLog.xlsx File Exsists,moving onto Reading and relabeling
Time to relabel the files and generate the new files
Location of folder: C:\Users\sitaparn\Desktop\FileRelabel\VAL-0149423- Process Parameter Study A Chromatography Runs - 2022-02-17
EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf 00_EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf

            Original
            Location : C:\Users\sitaparn\Desktop\FileRelabel\VAL-0149423- Process Parameter Study A Chromatography Runs - 2022-02-17\EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf
            Name : EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf

            Renamed
            Location: C:\Users\sitaparn\Desktop\FileRelabel\VAL-0149423- Process Parameter Study A Chromatography Ru

In [10]:
# df = pd.read_excel(scriptLogFileName)

In [11]:
# for number in df['PrefixIndex']:
#     print(f"Before: {number}")
#     print(f"After: {formatNumberToAppend(number)}")

In [12]:
# df["FormatedPreFix"] = df["PrefixIndex"].map(formatNumberToAppend)

In [13]:
# df["RenamedFileNames"] = df["FormatedPreFix"] +'_'+ df['OriginalFileName']

In [14]:
# df.head()

In [15]:
# locationValue = df['Location'][0]
# if str(locationValue) == 'nan':
#     location = os.getcwd()
# else:
#     location = df['Location'][0]

# print(f"Location of folder: {location}")

In [16]:
# folderName = str(df['FolderName'][0])
# try:
#     folderlocation = os.path.join(location,folderName)
#     os.mkdir(folderlocation)
# except OSError as error:
#     print(error)

In [17]:
# for pos in df.index:
# #     print(df.iloc[pos])
#     currentFileName = df["OriginalFileName"][pos]
#     renamedFileName = df["RenamedFileNames"][pos]
#     if str(renamedFileName) == 'nan':
#         print(f"Skipping {currentFileName}, since no PrefixIndex was assigned ")
#     else:    
#         print(currentFileName,renamedFileName)
#         src = os.path.join(os.getcwd(),currentFileName)
#         dst = os.path.join(location,folderName,renamedFileName)
#         shutil.copy(src,dst)
#         print(f"""
#         Original
#         Location : {src}
#         Name : {currentFileName}
        
#         Renamed
#         Location: {dst}
#         Name : {renamedFileName}
#         ###########################
#         """)


In [18]:
# def copyAndRename(original,loc,folderName,renamed):
#     cwd = os.getcwd()
#     originalInfo = os.path.join(cwd,original)
#     finalInfo = os.path.join(cwd,folderName,original)
#     print(f"COPY From {originalInfo}")
#     print(f"COPY TO {finalInfo}")
#     shutil.copy((originalInfo),str(finalInfo))

In [19]:
# copyAndRename(original,location,folderName,renamed)

In [20]:
# shutil.copy("EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf",
#             "New folder\\00_EXP20005199 VAL-0149423- Process Parameter Study A Chromatography Runs - Accepted at 2021-04-13T14.26.53Z.pdf")